In [1]:
import numpy as np
import netCDF4 as nc
from scipy import stats

In [2]:
def depth_weight_mean(z_i, var_vals):
    # Get index of last non-masked element at depth (counts provide the +1 for indexing interfaces)
    #print(var_vals.shape,'Meep')
    if len(var_vals.shape)==3:
        last_int_idx = np.ma.count(var_vals,axis=0)
    elif len(var_vals.shape)==4:
        last_int_idx = np.ma.count(var_vals,axis=1)
    last_depth = z_i[last_int_idx]
    
    # limit if deeper values occur than integration depth (e.g., 200m)
    # last_depth is a 2D array that will serve as a denominator for the layer thickness array
   
    int_max=mod_vars[var]['int_max']
    int_min=mod_vars[var]['int_min']
    
    last_depth[last_depth>int_max]=int_max
    last_depth[last_depth<int_min]=int_min
    # Create "layer_width" Object From Interface-Depth Array
    tmp_dep_i = z_i
    # Making all values below the integration later equal to int_depth will make the difference = 0
    # and, subsequently, the weight will be zero. This will not make the weight zero for areas below the bottom, though
    # for locations that are shallower than the int_depth. 
    tmp_dep_i[tmp_dep_i>int_max]=int_max
    tmp_dep_i[tmp_dep_i<int_min]=int_min
    # difference the array to get layer thickness
    lay_thick = np.diff(tmp_dep_i)
    
    # Broadcast to #dimensions of the 2D "max_depth"
    # NOTE: weights this will sum to more than 1 in places where depth is < int_depth. 
    #       HOWEVER, areas below bottom will be multipled against masked values and not counted in final weighted mean
    if len(var_vals.shape)==3:
        weights = lay_thick[:,None,None]/last_depth
        
    elif len(var_vals.shape)==4:
        weights = np.transpose(lay_thick[:,None,None,None]/last_depth, (1,0,2,3)) 
        
    # MULTIPLY the weights by the temp. 
    weighted_temp = weights*var_vals
    
    if len(var_vals.shape)==3:
        weighted_mean = np.ma.sum(weighted_temp,axis=0)
    elif len(var_vals.shape)==4:
        weighted_mean = np.ma.sum(weighted_temp,axis=1)
    
    return weighted_mean

In [3]:
e1n_dir = '/work/Liz.Drenkard/Fe_deposition_study/'

mod_vars = {#'nsmz_100':{'3D':False},
            #'nmdz_100':{'3D':False},
            #'nlgz_100':{'3D':False},
            #'chlos':{'3D':False}, 
            #'nsmp_100':{'3D':False}, 
            #'nlgp_100':{'3D':False}, 
            #'ndi_100':{'3D':False}
            #'dfe':{'3D':True}, 
            #'no3':{'3D':True},
            #'thetao':{'3D':True},
            #'so':{'3D':True},}
            #'pr':{'3D':False}, 
            #'sm_dust_col':{'3D':False},
            #'lg_dust_col':{'3D':False},
            #'wetdust':{'3D':False},
            #'drydust':{'3D':False},
            #'dep_wet_fed':{'3D':False},
            #'dep_dry_fed':{'3D':False},
            #'intpp':{'3D':False}, 
            #'epc100':{'3D':False},
            #'o2':{'3D':True,'int_min':100, 'int_max':600},
            #'zooc':{'3D':True,'int_min':0, 'int_max':100},
            #'pr':{'3D':False}
            #'mrsos':{'3D':False}
            'expc':{'3D':False}
    

}

dust_sim = {'static':{'dep_var': 'z_i','title': 'Static Dust'},}
            #'dynamic':{'dep_var':'lev_bnds','title': 'Dynamic Dust'}}

## For 2D Variables - including O2

In [4]:
# Only run to save out statistics 
import pickle

for var in mod_vars:
    print(var)
    for dsim in dust_sim:
    #for dsim in ['dynamic']:
        var_his = None
        dep = None
        
        pifil = e1n_dir + dsim + '/ocean_bgc_ts/' + var + '_' + '0101-0351_piC.nc'
        #pifil = e1n_dir + dsim + '/ocean_bgc_means/' + var + '_' + '0101-0351_piC.nc'#_ts.nc'
        #pifil = e1n_dir + dsim + '/fe_depostion_mechanism_fields/' + var + '_' + '0101-0351_piC_ts.nc'
        
        fid = nc.Dataset(pifil)       
            
        var_val = fid.variables[var][:].squeeze()
        print(var_val.shape)
        
        tim = np.arange(var_val.shape[0])
        
        if mod_vars[var]['3D']:
            # Looking at depth integrated o2 regression
            dep = fid.variables[dust_sim[dsim]['dep_var']][:].squeeze()
            if dep.ndim>1:
                dep = np.append(dep[:,0],dep[-1,-1])
            var_val = depth_weight_mean(dep, var_val)
                
        print(var_val.shape)
        
        # Define storage variables
        slope = np.zeros(var_val.shape[1:])
        r_value = np.zeros(var_val.shape[1:])
        p_value = np.zeros(var_val.shape[1:])
        
        for nr in np.arange(var_val.shape[1]):
            for ni in np.arange(var_val.shape[2]):
                slope[nr,ni], _, r_value[nr,ni], p_value[nr,ni], _ = stats.linregress(tim,var_val[:,nr,ni])
       
        # SAVE values to file
        pickle_fil = (var + '_' + dsim + '_' + 'piC_0101-0351_stats.pkl')
        
        with open(pickle_fil, 'wb') as f: 
            pickle.dump([slope, p_value], f)
            

expc
(3012, 180, 360)
(3012, 180, 360)


## For 3D O2

In [6]:
# Only run to save out statistics 
import pickle

#mod_vars = {'o2':{'3D':True,'int_min':100, 'int_max':600}}
mod_vars = {'thetao':{'3D':True,'int_min':100, 'int_max':600}}
#mod_vars = {'zooc':{'3D':True,'int_min':100, 'int_max':600}}
#mod_vars = {'phydiat':{'3D':True,'int_min':0, 'int_max':600},
#            'phydiaz':{'3D':True,'int_min':0, 'int_max':600},
#            'phymisc':{'3D':True,'int_min':0, 'int_max':600},
#            'phypico':{'3D':True,'int_min':0, 'int_max':600}}


for var in mod_vars:
    print(var)
    for dsim in dust_sim:
        var_his = None
        dep = None
        
        pifil = e1n_dir + dsim + '/ocean_bgc_ts/' + var + '_' + '0101-0351_piC_ts.nc'
        #pifil = e1n_dir + dsim + '/ocean_bgc_means/' + var + '_' + '0101-0351_piC_ts.nc'
        #pifil = e1n_dir + dsim + '/ocean_bgc_ts/' + var + '0101-0351_piC_ts.nc'
        
        fid = nc.Dataset(pifil)       
            
        var_val = fid.variables[var][:].squeeze()
        print(var_val.shape)
        
        tim = np.arange(var_val.shape[0])
        
        # Define storage variables
        slope = np.zeros(var_val.shape[1:])
        r_value = np.zeros(var_val.shape[1:])
        p_value = np.zeros(var_val.shape[1:])
        
        if mod_vars[var]['3D']:
                
            for nd in np.arange(var_val.shape[1]):
                for nr in np.arange(var_val.shape[2]):
                    for ni in np.arange(var_val.shape[3]):
                        slope[nd,nr,ni], _, r_value[nd,nr,ni], p_value[nd,nr,ni], _ = stats.linregress(tim,var_val[:,nd,nr,ni])
        else:
        
            for nr in np.arange(var_val.shape[1]):
                for ni in np.arange(var_val.shape[2]):
                    slope[nr,ni], _, r_value[nr,ni], p_value[nr,ni], _ = stats.linregress(tim,var_val[:,nr,ni])
       
        # SAVE values to file
        pickle_fil = (var + '_' + dsim + '_' + 'piC_0101-0351_stats.pkl')
        
        with open(pickle_fil, 'wb') as f: 
            pickle.dump([slope, p_value], f)
            
            

thetao
(3012, 35, 180, 360)
(3012, 35, 180, 360)


## 3D variables

In [9]:
# Only run to save out statistics 
import pickle

mod_vars = {'thetao':{'3D':True},
            'so':{'3D':True},}

for var in mod_vars:
    print(var)
    for dsim in dust_sim:
        var_his = None
        dep = None
        
        pifil = e1n_dir + dsim + '/ocean_bgc_ts/' + var + '_0101-0351_piC_ts.nc'
        
        fid = nc.Dataset(pifil)       
            
        var_val = fid.variables[var][:].squeeze()
        print(var_val.shape)
        
        tim = np.arange(var_val.shape[0])
        
        # Define storage variables
        slope = np.zeros(var_val.shape[1:])
        r_value = np.zeros(var_val.shape[1:])
        p_value = np.zeros(var_val.shape[1:])
        
        if mod_vars[var]['3D']:
                
            for nd in np.arange(var_val.shape[1]):
                for nr in np.arange(var_val.shape[2]):
                    for ni in np.arange(var_val.shape[3]):
                        slope[nd,nr,ni], _, r_value[nd,nr,ni], p_value[nd,nr,ni], _ = stats.linregress(tim,var_val[:,nd,nr,ni])
        else:
        
            for nr in np.arange(var_val.shape[1]):
                for ni in np.arange(var_val.shape[2]):
                    slope[nr,ni], _, r_value[nr,ni], p_value[nr,ni], _ = stats.linregress(tim,var_val[:,nr,ni])
       
        # SAVE values to file
        pickle_fil = (var + '_' + dsim + '_' + 'piC_0101-0351_stats.pkl')
        
        with open(pickle_fil, 'wb') as f: 
            pickle.dump([slope, p_value], f)
            
            

thetao
(3012, 35, 180, 360)
(3012, 35, 180, 360)
so
(3012, 35, 180, 360)
(3012, 35, 180, 360)
